# Tutorial: Mixer Unit Model with Ideal Property Package


![](mixer.svg)

**Problem Statement**: Consider the process of mixing benzene and toluene streams to form a mixture with the following inlet specificatons:

**Stream 1:**

Benzene Flow Rate = 0.6 kmol/hr

Pressure = 101325 Pa 

Temperature = 353 K

**Stream 2**

Toluene Flow Rate = 0.4 kmol/hr

Pressure = 202650 Pa 

Temperature = 356 K

We will study the simulation of the mixer with the given specifications using 2 approaches:

* Case 1: Specifying the number of mixer inlets, and momentum mixing type set to 'minimize'

* Case 2: Specifying the mixer inlet names, and momentum mixing type set to 'equality' (only one inlet stream pressure    specified)

**Note: 
When the momentum mixing type is set to 'minimize', the mixed stream pressure takes the minimum value among all inlet stream pressures.
When the momentum mixing type is set to 'equality', the mixed stream, along with all inlet streams have the same value of pressure.**


For more details, IDAES documentation reference for mixer model: https://idaes-pse.readthedocs.io/en/stable/models/mixer.html

## Setting up the problem in IDAES

In [1]:
# Import objects from pyomo package 
from pyomo.environ import ConcreteModel, SolverFactory, value

# Import the main FlowsheetBlock from IDAES. The flowsheet block will contain the unit model
from idaes.core import FlowsheetBlock

# Import idaes logger to set output levels
import idaes.logger as idaeslog

#Create the ConcreteModel and the FlowsheetBlock, and attach the flowsheet block to it.
m = ConcreteModel()

m.fs = FlowsheetBlock(default={"dynamic": False}) # dynamic or ss flowsheet needs to be specified here

#import the BTX_ideal property package to create a properties block for the flowsheet
from idaes.generic_models.properties.activity_coeff_models import BTX_activity_coeff_VLE

# Add properties parameter block to the flowsheet with specifications
m.fs.properties = BTX_activity_coeff_VLE.BTXParameterBlock(default={"valid_phase":
                                                     'Liq',
                                                     "activity_coeff_model":
                                                     "Ideal"})
#Import the mixer unit model from the model library
from idaes.generic_models.unit_models.mixer import Mixer

#Import momentum mixing type
from idaes.generic_models.unit_models.mixer import MomentumMixingType

## Case 1

In [2]:
#Create an instance of the mixer unit, attaching it to the flowsheet
#Specify that the property package to be used with the mixer is the one we created earlier, the number of mixer inlets is 2, and momentum mixing type is minimize
m.fs.mixer1 = Mixer(default={"property_package": m.fs.properties,"num_inlets":2,"momentum_mixing_type":MomentumMixingType.minimize})

#Import the degrees_of_freedom function from the idaes.core.util.model_statistics package
# DOF = Number of Model Variables - Number of Model Constraints
from idaes.core.util.model_statistics import degrees_of_freedom

# Call the degrees_of_freedom function, get initial DOF
DOF_initial = degrees_of_freedom(m)
print('The initial degrees of freedom is: {0}'.format(DOF_initial))

The initial degrees of freedom is: 10


In [3]:
assert DOF_initial == 10

In [4]:
#Fix the stream inlet conditions
m.fs.mixer1.inlet_1.flow_mol.fix(0.6*1000/3600) # converting to mol/s as unit basis is mol/s
m.fs.mixer1.inlet_1.mole_frac_comp[0, "benzene"].fix(1)
m.fs.mixer1.inlet_1.mole_frac_comp[0, "toluene"].fix(0)
m.fs.mixer1.inlet_1.pressure.fix(101325) # Pa
m.fs.mixer1.inlet_1.temperature.fix(353) # K
m.fs.mixer1.inlet_2.flow_mol.fix(0.4*1000/3600) # converting to mol/s as unit basis is mol/s
m.fs.mixer1.inlet_2.mole_frac_comp[0, "benzene"].fix(0)
m.fs.mixer1.inlet_2.mole_frac_comp[0, "toluene"].fix(1)
m.fs.mixer1.inlet_2.pressure.fix(202650) # Pa
m.fs.mixer1.inlet_2.temperature.fix(356) # K

# Call the degrees_of_freedom function, get final DOF
DOF_final = degrees_of_freedom(m)
print('The final degrees of freedom is: {0}'.format(DOF_final))

The final degrees of freedom is: 0


In [5]:
assert DOF_final == 0

### Flowsheet Initialization

In [6]:
#Initialize the flowsheet, and set the output at WARNING
m.fs.mixer1.initialize(outlvl=idaeslog.WARNING)

### Obtaining Simulation Results

In [7]:
#Solve the simulation using ipopt
#Note: If the degrees of freedom = 0, we have a square problem
opt = SolverFactory('ipopt')
result = opt.solve(m, tee=True)

Ipopt 3.13.2: 

******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit http://projects.coin-or.org/Ipopt

This version of Ipopt was compiled from source code available at
    https://github.com/IDAES/Ipopt as part of the Institute for the Design of
    Advanced Energy Systems Process Systems Engineering Framework (IDAES PSE
    Framework) Copyright (c) 2018-2019. See https://github.com/IDAES/idaes-pse.

This version of Ipopt was compiled using HSL, a collection of Fortran codes
    for large-scale scientific computation.  All technical papers, sales and
    publicity material resulting from use of the HSL codes within IPOPT must
    contain the following acknowledgement:
        HSL, a collection of Fortran codes for large-scale scientific
        computation. See http://

In [8]:
from pyomo.opt import TerminationCondition, SolverStatus

# Check if termination condition is optimal
assert result.solver.termination_condition == TerminationCondition.optimal
assert result.solver.status == SolverStatus.ok

### View Results

In [9]:
#Display the outlet stream mole flowrate (mol/s)
m.fs.mixer1.outlet.flow_mol[0].display()

#Display the outlet stream molar enthalpy (J/mol)
m.fs.mixer1.mixed_state[0].enth_mol_phase.display()

flow_mol : Total molar flowrate [mol/s]
    Size=1, Index=None
    Key  : Lower : Value              : Upper : Fixed : Stale : Domain
    None :     0 : 0.2777777777777778 :  None : False : False : NonNegativeReals
enth_mol_phase : Phase molar specific enthalpies [J/mol]
    Size=1, Index=fs.properties.phase_list
    Key : Lower : Value              : Upper : Fixed : Stale : Domain
    Liq :  None : 42703.868530442516 :  None : False : False :  Reals


In [10]:
#Display a readable result report
m.fs.mixer1.report()


Unit : fs.mixer1                                                           Time: 0.0
------------------------------------------------------------------------------------
    Stream Table
                             inlet_1    inlet_2    Outlet  
    flow_mol                  0.16667    0.11111    0.27778
    mole_frac_comp benzene     1.0000     0.0000    0.60000
    mole_frac_comp toluene     0.0000     1.0000    0.40000
    temperature                353.00     356.00     354.31
    pressure               1.0132e+05 2.0265e+05 1.0132e+05


In [11]:
#Energy Balance Validation 

molflow_1 = m.fs.mixer1.inlet_1.flow_mol[0].value
molflow_2 = m.fs.mixer1.inlet_2.flow_mol[0].value
benzene_1 = m.fs.mixer1.inlet_1.mole_frac_comp[0, "benzene"].value
toluene_2 = m.fs.mixer1.inlet_2.mole_frac_comp[0, "toluene"].value

#total_enthalpy_in = molflow_1*benzene_1*molar_enthalpy_1 + molflow_2*toluene_2*molar_enthalpy_2
total_enthalpy_in1 = molflow_1*m.fs.mixer1.inlet_1_state[0].enth_mol_phase['Liq'].value
total_enthalpy_in2 = molflow_2*m.fs.mixer1.inlet_2_state[0].enth_mol_phase['Liq'].value
print('The total enthalpy of inlet streams is {0} J/s'.format(total_enthalpy_in1 + total_enthalpy_in2))

molar_enthalpy_out = m.fs.mixer1.mixed_state[0].enth_mol_phase['Liq'].value
mole_flow_out = m.fs.mixer1.outlet.flow_mol[0].value
total_enthalpy_out = mole_flow_out*molar_enthalpy_out
print('The total enthalpy of the outlet stream is {0} J/s'.format(total_enthalpy_out))

The total enthalpy of inlet streams is 11862.1857029007 J/s
The total enthalpy of the outlet stream is 11862.185702900699 J/s


In [12]:
import pytest

# Check results
assert mole_flow_out == pytest.approx(0.27778, abs=1e-2)
assert m.fs.mixer1.outlet.mole_frac_comp[0, "benzene"].value == pytest.approx(0.6, abs=1e-3)
assert m.fs.mixer1.outlet.mole_frac_comp[0, "toluene"].value == pytest.approx(0.4, abs=1e-3)
assert m.fs.mixer1.outlet.temperature[0].value == pytest.approx(354.31, abs=1e-2)
assert m.fs.mixer1.outlet.pressure[0].value == pytest.approx(101325, abs=1)
assert total_enthalpy_out - total_enthalpy_in1 - total_enthalpy_in2 == pytest.approx(0, abs=1e-2)

## Case 2

In [13]:
#Create the ConcreteModel and the FlowsheetBlock, and attach the flowsheet block to it.
m1 = ConcreteModel()
m1.fs = FlowsheetBlock(default={"dynamic": False})

# Add properties parameter block to the flowsheet with specifications
m1.fs.properties = BTX_activity_coeff_VLE.BTXParameterBlock(default={"valid_phase":
                                                     'Liq',
                                                     "activity_coeff_model":
                                                     "Ideal"})

In [14]:
#Create an instance of another mixer unit, attaching it to the flowsheet
#Specify that the property package to be used with the mixer is the one we created earlier, inlet list is specified, and momentum mixing type is equality
m1.fs.mixer2 = Mixer(default={"property_package": m1.fs.properties,"inlet_list":["inlet1","inlet2"],"momentum_mixing_type":MomentumMixingType.equality})

In [15]:
DOF_init = degrees_of_freedom(m1)
print('The initial degrees of freedom is: {0}'.format(DOF_init))
#Fix the stream inlet conditions
m1.fs.mixer2.inlet1.flow_mol.fix(0.6*1000/3600) # converting to mol/s as unit basis is mol/s
m1.fs.mixer2.inlet1.mole_frac_comp[0, "benzene"].fix(1)
m1.fs.mixer2.inlet1.mole_frac_comp[0, "toluene"].fix(0)
m1.fs.mixer2.inlet1.pressure.fix(101325) # Pa , Another option is m1.fs.mixer2.inlet2.pressure.fix(202650)
m1.fs.mixer2.inlet1.temperature.fix(353) # K
m1.fs.mixer2.inlet2.flow_mol.fix(0.4*1000/3600) # converting to mol/s as unit basis is mol/s
m1.fs.mixer2.inlet2.mole_frac_comp[0, "benzene"].fix(0)
m1.fs.mixer2.inlet2.mole_frac_comp[0, "toluene"].fix(1)
m1.fs.mixer2.inlet2.temperature.fix(356) # K
DOF_final = degrees_of_freedom(m1)
print('The final degrees of freedom is: {0}'.format(DOF_final))

The initial degrees of freedom is: 9
The final degrees of freedom is: 0


### Flowsheet Initialization

In [16]:
#Initialize the flowsheet, and set the output at WARNING
m1.fs.mixer2.initialize(outlvl=idaeslog.WARNING)

### Obtaining Simulation Results

In [17]:
#Solve the simulation using ipopt
#Note: If the degrees of freedom = 0, we have a square problem
opt = SolverFactory('ipopt')
result = opt.solve(m1)

In [18]:
from pyomo.opt import TerminationCondition, SolverStatus

# Check if termination condition is optimal
assert result.solver.termination_condition == TerminationCondition.optimal
assert result.solver.status == SolverStatus.ok

### View Results

In [19]:
#Display the outlet stream mole flowrate (mol/s)
m1.fs.mixer2.outlet.flow_mol[0].display()

#Display the outlet stream molar enthalpy (J/mol)
m1.fs.mixer2.mixed_state[0].enth_mol_phase.display()

flow_mol : Total molar flowrate [mol/s]
    Size=1, Index=None
    Key  : Lower : Value              : Upper : Fixed : Stale : Domain
    None :     0 : 0.2777777777777778 :  None : False : False : NonNegativeReals
enth_mol_phase : Phase molar specific enthalpies [J/mol]
    Size=1, Index=fs.properties.phase_list
    Key : Lower : Value              : Upper : Fixed : Stale : Domain
    Liq :  None : 42703.868530442516 :  None : False : False :  Reals


In [20]:
#Display a readable report
m1.fs.mixer2.report()


Unit : fs.mixer2                                                           Time: 0.0
------------------------------------------------------------------------------------
    Stream Table
                             inlet1     inlet2     Outlet  
    flow_mol                  0.16667    0.11111    0.27778
    mole_frac_comp benzene     1.0000     0.0000    0.60000
    mole_frac_comp toluene     0.0000     1.0000    0.40000
    temperature                353.00     356.00     354.31
    pressure               1.0132e+05 1.0132e+05 1.0132e+05


In [21]:
# Check results
assert m1.fs.mixer2.outlet.flow_mol[0].value == pytest.approx(0.27778, abs=1e-2)
assert m1.fs.mixer2.outlet.mole_frac_comp[0, "benzene"].value == pytest.approx(0.6, abs=1e-3)
assert m1.fs.mixer2.outlet.mole_frac_comp[0, "toluene"].value == pytest.approx(0.4, abs=1e-3)
assert m1.fs.mixer2.outlet.temperature[0].value == pytest.approx(354.31, abs=1e-2)
assert m1.fs.mixer2.outlet.pressure[0].value == pytest.approx(101325, abs=1)